# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 14 2023 11:13AM,257342,10,MSP218530,"Methapharm, Inc.",Released
1,Feb 14 2023 11:13AM,257342,10,MSP218531,"Methapharm, Inc.",Released
2,Feb 14 2023 11:13AM,257342,10,MSP218535,"Methapharm, Inc.",Released
3,Feb 14 2023 11:13AM,257342,10,MSP218533,Methapharm-G,Released
4,Feb 14 2023 11:13AM,257342,10,MSP218534,Methapharm-G,Released
5,Feb 14 2023 11:11AM,257341,10,0086379097,ISDIN Corporation,Released
6,Feb 14 2023 11:11AM,257341,10,0086379140,ISDIN Corporation,Released
7,Feb 14 2023 11:07AM,257340,10,PRF-52920,Bio-PRF,Released
8,Feb 14 2023 11:07AM,257340,10,PRF-52923,Bio-PRF,Released
9,Feb 14 2023 11:07AM,257340,10,PRF-52927,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,257338,Released,1
43,257339,Released,1
44,257340,Released,44
45,257341,Released,2
46,257342,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257338,NaN,1.0
257339,NaN,1.0
257340,NaN,44.0
257341,NaN,2.0
257342,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257254,0.0,1.0
257256,0.0,4.0
257258,0.0,1.0
257259,0.0,1.0
257263,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257254,0,1
257256,0,4
257258,0,1
257259,0,1
257263,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257254,0,1
1,257256,0,4
2,257258,0,1
3,257259,0,1
4,257263,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257254,,1
1,257256,,4
2,257258,,1
3,257259,,1
4,257263,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 14 2023 11:13AM,257342,10,"Methapharm, Inc."
3,Feb 14 2023 11:13AM,257342,10,Methapharm-G
5,Feb 14 2023 11:11AM,257341,10,ISDIN Corporation
7,Feb 14 2023 11:07AM,257340,10,Bio-PRF
51,Feb 14 2023 10:56AM,257339,19,"GCH Granules USA, Inc."
52,Feb 14 2023 10:44AM,257338,20,Else Nutrition
53,Feb 14 2023 10:38AM,257336,20,"HVL, LLC dba Atrium Innovations"
54,Feb 14 2023 10:27AM,257334,10,"CLINUVEL, Inc."
55,Feb 14 2023 10:24AM,257333,16,"SHL Pharma, LLC"
56,Feb 14 2023 10:10AM,257331,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 14 2023 11:13AM,257342,10,"Methapharm, Inc.",,5
1,Feb 14 2023 11:13AM,257342,10,Methapharm-G,,5
2,Feb 14 2023 11:11AM,257341,10,ISDIN Corporation,,2
3,Feb 14 2023 11:07AM,257340,10,Bio-PRF,,44
4,Feb 14 2023 10:56AM,257339,19,"GCH Granules USA, Inc.",,1
5,Feb 14 2023 10:44AM,257338,20,Else Nutrition,,1
6,Feb 14 2023 10:38AM,257336,20,"HVL, LLC dba Atrium Innovations",,1
7,Feb 14 2023 10:27AM,257334,10,"CLINUVEL, Inc.",,1
8,Feb 14 2023 10:24AM,257333,16,"SHL Pharma, LLC",,1
9,Feb 14 2023 10:10AM,257331,10,ISDIN Corporation,,31


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 11:13AM,257342,10,"Methapharm, Inc.",5,
1,Feb 14 2023 11:13AM,257342,10,Methapharm-G,5,
2,Feb 14 2023 11:11AM,257341,10,ISDIN Corporation,2,
3,Feb 14 2023 11:07AM,257340,10,Bio-PRF,44,
4,Feb 14 2023 10:56AM,257339,19,"GCH Granules USA, Inc.",1,
5,Feb 14 2023 10:44AM,257338,20,Else Nutrition,1,
6,Feb 14 2023 10:38AM,257336,20,"HVL, LLC dba Atrium Innovations",1,
7,Feb 14 2023 10:27AM,257334,10,"CLINUVEL, Inc.",1,
8,Feb 14 2023 10:24AM,257333,16,"SHL Pharma, LLC",1,
9,Feb 14 2023 10:10AM,257331,10,ISDIN Corporation,31,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 11:13AM,257342,10,"Methapharm, Inc.",5,
1,Feb 14 2023 11:13AM,257342,10,Methapharm-G,5,
2,Feb 14 2023 11:11AM,257341,10,ISDIN Corporation,2,
3,Feb 14 2023 11:07AM,257340,10,Bio-PRF,44,
4,Feb 14 2023 10:56AM,257339,19,"GCH Granules USA, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 11:13AM,257342,10,"Methapharm, Inc.",5.0,NaN
1,Feb 14 2023 11:13AM,257342,10,Methapharm-G,5.0,NaN
2,Feb 14 2023 11:11AM,257341,10,ISDIN Corporation,2.0,NaN
3,Feb 14 2023 11:07AM,257340,10,Bio-PRF,44.0,NaN
4,Feb 14 2023 10:56AM,257339,19,"GCH Granules USA, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 11:13AM,257342,10,"Methapharm, Inc.",5.0,0.0
1,Feb 14 2023 11:13AM,257342,10,Methapharm-G,5.0,0.0
2,Feb 14 2023 11:11AM,257341,10,ISDIN Corporation,2.0,0.0
3,Feb 14 2023 11:07AM,257340,10,Bio-PRF,44.0,0.0
4,Feb 14 2023 10:56AM,257339,19,"GCH Granules USA, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5918139,245.0,24.0
12,514641,0.0,2.0
15,1029092,22.0,42.0
16,257333,1.0,0.0
19,2058369,69.0,5.0
20,771966,2.0,1.0
21,514596,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5918139,245.0,24.0
1,12,514641,0.0,2.0
2,15,1029092,22.0,42.0
3,16,257333,1.0,0.0
4,19,2058369,69.0,5.0
5,20,771966,2.0,1.0
6,21,514596,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,245.0,24.0
1,12,0.0,2.0
2,15,22.0,42.0
3,16,1.0,0.0
4,19,69.0,5.0
5,20,2.0,1.0
6,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,245.0
1,12,Released,0.0
2,15,Released,22.0
3,16,Released,1.0
4,19,Released,69.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,24.0,2.0,42.0,0.0,5.0,1.0,0.0
Released,245.0,0.0,22.0,1.0,69.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,24.0,2.0,42.0,0.0,5.0,1.0,0.0
1,Released,245.0,0.0,22.0,1.0,69.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,24.0,2.0,42.0,0.0,5.0,1.0,0.0
1,Released,245.0,0.0,22.0,1.0,69.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()